In [ ]:
import os
import csv
import warnings
import numpy as np
from glob import glob
from pyplanter import PlantedH5
from lib import metrics

In [ ]:
"""System paths for reading and exporting data. Currently does not support more then one directory at once.

input_path (str): Path to directory with *.out data. Requires non-nested directory structure.
output_path (str): Path to export directory. Source directory will be created here.
file_name (str): Single file export. Leave empty '' for batch (within directory) processing.
"""

input_path = '/backup/data/muni/Cells/test_data/'
output_path = '/backup/data/muni/Cells/Export/'
output_file = 'output2.csv'
file_name = None

# ----------------------- Notebook global settings -------------------------
# Name of peak markers
VALID_MARK_LABELS = {'Peak detection'}
# Smooth beat series with windows size 1-n, default: 0
SMOOTHER = 0
# Analysis to use: 'sdnn', 'sdsd', 'rmssd', 'psd', 'poincare', 'sample_entropy'
VALID_METHODS = ['sdnn', 'sdsd', 'rmssd', 'psd', 'poincare', 'sample_entropy']

VALID_LIM = 4

In [ ]:
""" Output format settings. Variables `columns`, `custom_channel_names`, `custom_channel_units`, `custom_multipliers`
can be specified as empty tuple `()`. In this case default values are inherited from *.ou file. Is specified, all tuples
must contain the same number of elements.

output_format (str): 'h5' or 'csv' are valid values
columns (tuple): Select specific channels only
custom_channel_names (tuple): Custom name for each channel
custom_channel_units (tuple): New units for each channel
custom_multipliers (tuple): Converts each channel to new units/range by using custom multiplier (currently not working for csv)

"""
# list files to process
file_list = glob(os.path.join(input_path, '**', file_name if file_name else '*.h5'), recursive=True)

planter = PlantedH5()
metrics = metrics.Metrics(smooth=SMOOTHER)

processed_files = list()
for file_path in file_list:

    file_name = os.path.basename(file_path)
    print(f'Processing file {file_name}.')

    # create file object
    planter.open(file_path)    

    if 'Marks' not in planter.f_obj:
        continue
    
    # read sampling frequency
    fs = planter.f_obj.attrs['Fs'][0]
    
    # read peak positions and convert it to ms
    peak_positions = [mark[0] * (1000 / fs) for mark in planter.f_obj['Marks'] if mark[-1].decode('UTF-8') in VALID_MARK_LABELS]
    peak_positions = sorted(peak_positions)
    peak_positions = np.array(peak_positions)
    nn = np.diff(peak_positions)

    # fast check
    nn_median = np.median(nn)
    below_lim = nn < ((1/VALID_LIM) * nn_median)
    above_lim = nn > (VALID_LIM * nn_median)
    if (
        np.any(below_lim) or
        np.any(above_lim)
    ):
        warnings.warn("NN values outside expected interval found. Replaced by median NN.")
        nn[below_lim] = nn_median
        nn[above_lim] = nn_median

    # compute metrics
    metrics.compute(nn, method_list=VALID_METHODS)

    processed_files.append(file_name)
    
    # close file handler
    planter.close()

# add new column with file ids
metrics.results['file_id'] = processed_files

if output_path and os.path.isdir(output_path):
    metrics.results.to_csv(os.path.join(output_path, output_file))

print('Job done!')
